In [1]:
import pandas as pd
import geopandas as gpd

import shared_utils
from utils import *
import intake
import gcsfs

import calitp
from calitp.tables import tbl
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
coverage_gdf = gpd.read_parquet(f'{GCS_FILE_PATH}block_accessibility_metrics.parquet')

In [3]:
coverage_gdf.head(3)

,county,tract,block,geo_id,geometry,block_pop,area,calitp_itp_id,rt_accessible,rt,accessible,static,rt_milestone
0,65,42628,4008,60650426284008,"POLYGON ((257831.965 -463783.472, 258008.040 -...",103,15986.958897,269,False,True,False,True,False
1,65,42711,3025,60650427113025,"POLYGON ((259577.302 -476173.973, 259651.066 -...",42,29609.218158,269,False,True,False,True,False
2,65,43401,1023,60650434011023,"POLYGON ((281412.631 -469351.296, 281613.718 -...",21,19971.850439,13,False,False,False,True,True


In [4]:
# popup_dict = {
#     "rt_accessible": "Accessible Trip and Stop, RT Feed Available",
#     "rt": "RT Feed Available",
#     "accessible": "Accessible Trip and Stop",
#     "static": "GTFS Static Present",
# }

popup_dict = {}

In [5]:
import branca

In [6]:
from shared_utils import calitp_color_palette as cp

In [7]:
cp.CALITP_CATEGORY_BRIGHT_COLORS

['#2EA8CE', '#EB9F3C', '#F4D837', '#51BF9D', '#8CBCCB', '#9487C0']

In [8]:
coverage_gdf.geometry = coverage_gdf.buffer(0)

In [9]:
df_rt_acc = (coverage_gdf.copy() >> filter(_.rt_accessible)).dissolve(by='calitp_itp_id').reset_index()
df_rt = (coverage_gdf.copy() >> filter(_.rt)).dissolve(by='calitp_itp_id').reset_index()
df_rt_milestone = (coverage_gdf.copy() >> filter(_.rt_milestone)).dissolve(by='calitp_itp_id').reset_index()
df_acc = (coverage_gdf.copy() >> filter(_.accessible)).dissolve(by='calitp_itp_id').reset_index()
df_static = (coverage_gdf.copy() >> filter(_.static)).dissolve(by='calitp_itp_id').reset_index()

for df in [df_rt_acc, df_rt, df_acc, df_static]:
    df['geometry'] = df.geometry.buffer(150).simplify(tolerance=75)

In [18]:
df_rt_milestone.head(3)

,calitp_itp_id,geometry,county,tract,block,geo_id,block_pop,area,rt_accessible,rt,accessible,static,rt_milestone
0,6,"MULTIPOLYGON (((-200957.368 -19276.968, -20083...",1,422000,1038,60014220001038,11,7210.455283,False,False,False,True,True
1,13,"MULTIPOLYGON (((-326996.871 236622.306, -32712...",65,43401,1023,60650434011023,21,19971.850439,False,False,False,True,True
2,14,"MULTIPOLYGON (((184574.222 -462499.383, 184558...",59,88403,1002,60590884031002,256,19695.690969,False,False,False,True,True


In [12]:
popup_dict = {"calitp_itp_id": 'ITP ID',
            "rt_accessible": "Accessible Trip and Stop, RT Feed Available",
            "rt": "RT Feed Available",
            "rt_milestone": "RT Feed in MSD Milestones",
            "accessible": "Accessible Trip and Stop",
            "static": "GTFS Static Present",
             }

In [19]:
LAYERS_DICT = {
    "RT Accessible": {"df": df_rt_acc,
        "plot_col": "rt_accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[:1],
    },
    "RT": {"df": df_rt,
        "plot_col": "rt",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[4:5],
    },
    "RT Milestones": {"df": df_rt_milestone,
        "plot_col": "rt",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[1:2],
    },
    "Accessible": {"df": df_acc,
        "plot_col": "accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[2:3],
    },
    "GTFS Static": {"df": df_static,
        "plot_col": "static",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[3:4],
    }
}

In [20]:
# shared_utils.map_utils.make_folium_multiple_layers_map?

In [21]:
fig = shared_utils.map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 800, fig_height = 800, 
        zoom=9, 
        centroid = [36.5, -120], 
        title=f"Statewide Feed Coverage",
        # legend_dict = LEGEND_DICT
    )

In [24]:
# fig

In [23]:
fig.save('statewide_coverage_map.html')